# TITLE: 

# By 
## Name1:
## Name2:
## Name3:

In [13]:
## Libaries
import pandas as pd
import numpy as np
import plotly.express as px

In [28]:
# Files
API_GameLogs = 'gameLogs.csv'
data = pd.read_csv(API_GameLogs)\

data.head()


,Unnamed: 0,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,...,GAME_DATE,GAME_ID,HOME_FLAG,AWAY_FLAG,HOME_WIN_PCTG,AWAY_WIN_PCTG,TOTAL_WIN_PCTG,ROLLING_SCORING_MARGIN,ROLLING_OE,NUM_REST_DAYS
0,0,Atlanta,Hawks,1610612737,1,0,1,0,0,0,...,2021-05-16,22001066,1,0,0.694444,0.444444,0.569444,18.666667,0.594231,3.0
1,1,OPPONENTS,Rockets,1610612745,0,1,0,0,0,1,...,2021-05-16,22001066,0,1,0.250000,0.222222,0.236111,-8.000000,0.559973,2.0
2,2,Atlanta,Hawks,1610612737,1,0,1,0,0,0,...,2021-05-13,22001049,1,0,0.685714,0.444444,0.563380,9.333333,0.571405,1.0
3,3,OPPONENTS,Magic,1610612753,0,1,0,0,0,1,...,2021-05-13,22001049,0,1,0.305556,0.294118,0.300000,-22.333333,0.502823,2.0
4,4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,...,2021-05-12,22001042,1,0,0.676471,0.444444,0.557143,-0.666667,0.589472,2.0


# Basics Graphs

In [21]:
# Filter data for the Atlanta Hawks
hawks_data = data[data['NICKNAME'] == 'Hawks']

# Convert 'GAME_DATE' to datetime for better plotting
hawks_data['GAME_DATE'] = pd.to_datetime(hawks_data['GAME_DATE'])

# Sort the DataFrame by 'GAME_DATE' to ensure correct chronological plotting
hawks_data_sorted = hawks_data.sort_values(by='GAME_DATE')

# Plotting total win percentage over time, using the sorted DataFrame
fig = px.scatter(hawks_data_sorted, x='GAME_DATE', y='TOTAL_WIN_PCTG', title='Atlanta Hawks Total Win Percentage Over Time',
              labels={'TOTAL_WIN_PCTG': 'Total Win Percentage', 'GAME_DATE': 'Game Date'}, trendline='lowess')

fig.update_layout(xaxis_title='Game Date', yaxis_title='Total Win %', showlegend=False)
fig.show()


/var/folders/wj/_8fkb1x10xn9hxrq5p8pfh280000gn/T/ipykernel_26980/2460080567.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/apple/miniconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/Users/apple/miniconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old beh

In [22]:
# Scatter plot for shooting efficiency
fig = px.scatter(hawks_data_sorted, x='FGA', y='FG', trendline='ols', 
                 title='Field Goals Made vs. Attempted (Shooting Efficiency)',
                 labels={'FG': 'Field Goals Made', 'FGA': 'Field Goals Attempted'})

fig.update_layout(xaxis_title='Field Goals Attempted', yaxis_title='Field Goals Made', showlegend=False)
fig.show()


In [23]:
# Scatter plot for three-point shooting trends
fig = px.scatter(hawks_data_sorted, x='FG3A', y='FG3', size='FG3_PCT', trendline='ols',
                 title='Three-Point Field Goals Made vs. Attempted',
                 labels={'FG3': 'Three-Point Field Goals Made', 'FG3A': 'Three-Point Field Goals Attempted', 'FG3_PCT': 'Three-Point Percentage'})

fig.update_layout(xaxis_title='Three-Point Field Goals Attempted', yaxis_title='Three-Point Field Goals Made', showlegend=False)
fig.show()


In [26]:
# Line plot for total rebounds over time
fig = px.line(hawks_data_sorted, x='GAME_DATE', y='TOT_REB', 
              title='Total Rebounds Over Time',
              labels={'TOT_REB': 'Total Rebounds', 'GAME_DATE': 'Game Date'})

fig.update_layout(xaxis_title='Game Date', yaxis_title='Total Rebounds', showlegend=False)
fig.show()


/Users/apple/miniconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [27]:
# Scatter plot to observe the impact of rest days on points scored
fig = px.scatter(hawks_data_sorted, x='NUM_REST_DAYS', y='PTS', trendline='ols',
                 title='Impact of Rest Days on Points Scored',
                 labels={'NUM_REST_DAYS': 'Number of Rest Days', 'PTS': 'Points Scored'})

fig.update_layout(xaxis_title='Number of Rest Days', yaxis_title='Points Scored', showlegend=False)
fig.show()


In [30]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=hawks_data_sorted['GAME_DATE'], y=hawks_data_sorted['TOTAL_WIN_PCTG'], name="Total Win Percentage"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=hawks_data_sorted['GAME_DATE'], y=hawks_data_sorted['ROLLING_OE'], name="Offensive Efficiency"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(title_text="Total Win Percentage and Offensive Efficiency Over Time")

# Set x-axis title
fig.update_xaxes(title_text="Game Date")

# Set y-axes titles
fig.update_yaxes(title_text="Total Win Percentage", secondary_y=False)
fig.update_yaxes(title_text="Offensive Efficiency", secondary_y=True)

fig.show()


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['Unnamed: 0', 'CITY', 'NICKNAME', 'TEAM_ID', 'W', 'L', 'W_HOME', 'L_HOME', 'W_ROAD', 'L_ROAD', 'TEAM_TURNOVERS', 'TEAM_REBOUNDS', 'GP', 'GS', 'ACTUAL_MINUTES', 'ACTUAL_SECONDS', 'FG', 'FGA', 'FG_PCT', 'FG3', 'FG3A', 'FG3_PCT', 'FT', 'FTA', 'FT_PCT', 'OFF_REB', 'DEF_REB', 'TOT_REB', 'AST', 'PF', 'STL', 'TOTAL_TURNOVERS', 'BLK', 'PTS', 'AVG_REB', 'AVG_PTS', 'DQ', 'OFFENSIVE_EFFICIENCY', 'SCORING_MARGIN', 'SEASON', 'GAME_DATE', 'GAME_ID', 'HOME_FLAG', 'AWAY_FLAG', 'HOME_WIN_PCTG', 'AWAY_WIN_PCTG', 'TOTAL_WIN_PCTG', 'ROLLING_SCORING_MARGIN', 'ROLLING_OE', 'NUM_REST_DAYS'] but received: HOME_LAST_GAME_OE

# Model